In [8]:
%matplotlib inline
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.fftpack import dct
import matplotlib.pyplot as plt
from python_speech_features.base import mfcc
import seq2seq_model

In [9]:
root = './'
keep_prob = 0.8
max_output_length = 
rnn_size = 256
num_layers = 2
batch_size = 16

In [ ]:
out_file = tf.gfile.Open(root + 'wsj0/transcripts/wsj0/wsj0.trans')

def get_next_input():
    trans = out_file.readline()
    cont, file = trans.split('(')
    file = file[:-2]
    sample_rate, signal = scipy.io.wavfile.read(FileOpen(root + file.rstrip('\n'), 'rb'))
    X = onehot(list(cont))
    Y = mfcc(signal, sample_rate)

In [10]:
# A custom class inheriting tf.gfile.Open for providing seek with whence
class FileOpen(tf.gfile.Open):
    def seek(self, position, whence = 0):
        if (whence == 0):
            tf.gfile.Open.seek(self, position)
        elif (whence == 1):
            tf.gfile.Open.seek(self, self.tell() + position)
        else:
            raise FileError

In [12]:
vocab = np.asarray(list(" '+-.ABCDEFGHIJKLMNOPQRSTUVWXYZ_"))
vocab_to_int = {}

def onehot(x):
    x = np.asarray(x)
    return np.tile(x, (32, 1)).T == vocab # 32 = vocab length

In [17]:
# Make a graph and it's session
train_graph = tf.Graph()
train_session = tf.InteractiveSession(graph=train_graph)

# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    model_x = tf.placeholder()
    model_y = tf.placeholder()
    texts_lengths = tf.placeholder()
    summaries_lengths = tf.placeholder()

    model = seq2seq_model(input_data=model_x,
                          target_data=model_y,
                          keep_prob=keep_prob,
                          text_length=texts_lengths,
                          summary_length=summaries_lengths,
                          max_output_length=max_output_length,
                          vocab_size=len(vocab),
                          rnn_size=rnn_size,
                          num_layers=num_layers,
                          vocab_to_int=vocab_to_int,
                          batch_size=batch_size)
    
    with train_session as session():
        sess.run(tf.global_variables_initializer())
        for epoch_i in range(1, epochs+1):
            while ()

INFO:tensorflow:Creating AttentionSeq2Seq in mode=train
INFO:tensorflow:
AttentionSeq2Seq:
  attention.class: AttentionLayerBahdanau
  attention.params: {}
  bridge.class: seq2seq.models.bridges.ZeroBridge
  bridge.params: {}
  decoder.class: seq2seq.decoders.AttentionDecoder
  decoder.params: {}
  embedding.dim: 100
  embedding.init_scale: 0.04
  embedding.share: false
  encoder.class: seq2seq.encoders.BidirectionalRNNEncoder
  encoder.params: {}
  inference.beam_search.beam_width: 0
  inference.beam_search.choose_successors_fn: choose_top_k
  inference.beam_search.length_penalty_weight: 0.0
  optimizer.clip_embed_gradients: 0.1
  optimizer.clip_gradients: 5.0
  optimizer.learning_rate: 0.0001
  optimizer.lr_decay_rate: 0.99
  optimizer.lr_decay_steps: 100
  optimizer.lr_decay_type: ''
  optimizer.lr_min_learning_rate: 1.0e-12
  optimizer.lr_staircase: false
  optimizer.lr_start_decay_at: 0
  optimizer.lr_stop_decay_at: 2147483647
  optimizer.name: Adam
  optimizer.params: {}
  optimi

AttributeError: 'NoneType' object has no attribute 'path'